Importing libraries, modules and setting the display options

In [1]:
import pandas as pd
import numpy as np
import re 

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
import seaborn as sns

import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest, ttest_ind, norm, chi2_contingency, f_oneway, chi2_contingency, mannwhitneyu, kruskal, kendalltau

from sklearn.linear_model import LinearRegression 
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode

# -----------------------------------------------------------------------
# Gestión de los warnings
import warnings
warnings.filterwarnings("ignore")


# -----------------------------------------------------------------------
# Comprobando que el display sea el máximo para mostrar mejor el data frame
pd.set_option("display.max_rows", None)  # Cambia el número máximo de filas mostradas
pd.set_option("display.max_columns", None)  # Cambia el número máximo de columnas mostradas
pd.set_option("display.max_colwidth", None)  # Cambia el ancho máximo de la pantalla
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

Read CSV

In [2]:
df_online_food = pd.read_csv(r"C:\Users\yaelp\Desktop\Yael\Proyecto PowerBI OnlineFood Elena-Yael\onlinefoods.csv")

- Checking only columns

In [3]:
print(f"The qty of rows and columns are: {df_online_food .shape}")
print(f"The names of the columns are: {df_online_food .columns}")

The qty of rows and columns are: (388, 13)
The names of the columns are: Index(['Age', 'Gender', 'Marital Status', 'Occupation', 'Monthly Income', 'Educational Qualifications', 'Family size', 'latitude', 'longitude', 'Pin code', 'Output', 'Feedback', 'Unnamed: 12'], dtype='object')


- Columns Rename
    - Spaces 
    - mayus for minus
    

In [4]:
df_online_food.columns = [col.lower().replace(' ', '_') for col in df_online_food.columns]

Checking the types

In [5]:
print(df_online_food.head(3))

   age  gender marital_status occupation  monthly_income educational_qualifications  family_size  latitude  longitude  pin_code output   feedback unnamed:_12
0   20  Female         Single    Student       No Income              Post Graduate            4   12.9766    77.5993    560001    Yes   Positive         Yes
1   24  Female         Single    Student  Below Rs.10000                   Graduate            3   12.9770    77.5773    560009    Yes   Positive         Yes
2   22    Male         Single    Student  Below Rs.10000              Post Graduate            3   12.9551    77.6593    560017    Yes  Negative          Yes


Checking column's content 

In [13]:
print(df_online_food.dtypes)

age                             int64
gender                         object
marital_status                 object
occupation                     object
monthly_income                 object
educational_qualifications     object
family_size                     int64
latitude                      float64
longitude                     float64
pin_code                        int64
output                           bool
feedback                       object
dtype: object


In [6]:
print(df_online_food.isnull().sum())

age                           0
gender                        0
marital_status                0
occupation                    0
monthly_income                0
educational_qualifications    0
family_size                   0
latitude                      0
longitude                     0
pin_code                      0
output                        0
feedback                      0
unnamed:_12                   0
dtype: int64


In [7]:
for col in df_online_food.columns:
    print(col.upper())  # Imprime el nombre de la columna en mayúsculas
    for val in df_online_food[col].unique():
        count = df_online_food[df_online_food[col] == val].shape[0]
        print(f"{val}: {count}")
    print()  # Añade una línea en blanco para separar las columnas


AGE
20: 9
24: 50
22: 57
27: 21
23: 73
21: 23
28: 15
25: 52
32: 16
30: 9
31: 8
26: 35
18: 1
19: 4
33: 1
29: 14

GENDER
Female: 166
Male: 222

MARITAL_STATUS
Single: 268
Married: 108
Prefer not to say: 12

OCCUPATION
Student: 207
Employee: 118
Self Employeed: 54
House wife: 9

MONTHLY_INCOME
No Income: 187
Below Rs.10000: 25
More than 50000: 62
10001 to 25000: 45
25001 to 50000: 69

EDUCATIONAL_QUALIFICATIONS
Post Graduate: 174
Graduate: 177
Ph.D: 23
Uneducated: 2
School: 12

FAMILY_SIZE
4: 63
3: 117
6: 29
2: 101
5: 54
1: 24

LATITUDE
12.9766: 8
12.977: 36
12.9551: 6
12.9473: 2
12.985: 14
12.9299: 5
12.9828: 5
12.9854: 4
12.8988: 5
12.9438: 5
12.8893: 4
12.9783: 16
12.982: 6
13.0298: 2
12.9983: 1
12.9925: 6
12.9306: 6
12.9353: 1
12.9155: 1
13.0019: 7
12.9698: 8
12.9261: 11
12.9119: 2
12.9662: 3
12.9565: 1
13.0206: 8
12.9635: 6
13.0067: 2
12.8845: 12
13.0158: 7
12.9343: 8
13.0012: 4
12.9442: 5
13.0487: 8
12.9889: 5
12.9335: 2
13.102: 2
12.9048: 9
12.9337: 7
12.9037: 3
13.0289: 3
12.9561: 

- monthly_income is object while it should be float. 
    - checking column:

    **MONTHLY_INCOME**   
No Income: 187             ---> Do not change    
Below Rs.10000: 25         ---> Change to:  "Below 10.000 rupees"                 To facilitate reading   
More than 50000: 62        ---> Change to:  "More than 50.000 rupees"             To facilitate reading   
10001 to 25000: 45         ---> Change to:  "Between 10.001 and 25.000 rupees"    To facilitate reading   
25001 to 50000: 69         ---> Change to:  "Between 25.001 and 50.000 rupees"    To facilitate reading   


Type must be: object

In [8]:
df_online_food['monthly_income'] = df_online_food['monthly_income'].replace({
    'No Income': 'No Income',
    'Below Rs.10000': 'Below 10.000 rupees',
    'More than 50000': 'More than 50.000 rupees',
    '10001 to 25000': 'Between 10.001 and 25.000 rupees',
    '25001 to 50000': 'Between 25.001 and 50.000 rupees'
})
df_online_food["monthly_income"].unique()

array(['No Income', 'Below 10.000 rupees', 'More than 50.000 rupees',
       'Between 10.001 and 25.000 rupees',
       'Between 25.001 and 50.000 rupees'], dtype=object)

- output column is object while it should be boolean
    - checking column:

    **OUTPUT**   
Yes: 301   
No: 87   

In [9]:
df_online_food['output'] = df_online_food['output'].replace({'Yes': True, 'No': False})
df_online_food["output"].dtype

dtype('bool')

- Do not know what unnamed:1_12 means
    - checking 

    Column has no sense, must be droped

In [10]:
df_online_food.drop(columns='unnamed:_12', inplace=True)

In [11]:
df_online_food.head()

,age,gender,marital_status,occupation,monthly_income,educational_qualifications,family_size,latitude,longitude,pin_code,output,feedback
0,20,Female,Single,Student,No Income,Post Graduate,4,12.9766,77.5993,560001,True,Positive
1,24,Female,Single,Student,Below 10.000 rupees,Graduate,3,12.9770,77.5773,560009,True,Positive
2,22,Male,Single,Student,Below 10.000 rupees,Post Graduate,3,12.9551,77.6593,560017,True,Negative
3,22,Female,Single,Student,No Income,Graduate,6,12.9473,77.5616,560019,True,Positive
4,22,Male,Single,Student,Below 10.000 rupees,Post Graduate,4,12.9850,77.5533,560010,True,Positive


In [14]:
print(df_online_food.dtypes)

age                             int64
gender                         object
marital_status                 object
occupation                     object
monthly_income                 object
educational_qualifications     object
family_size                     int64
latitude                      float64
longitude                     float64
pin_code                        int64
output                           bool
feedback                       object
dtype: object


In [16]:
df_online_food.describe()

,age,family_size,latitude,longitude,pin_code
count,388.000000,388.000000,388.000000,388.000000,388.000000
mean,24.628866,3.280928,12.972058,77.600160,560040.113402
std,2.975593,1.351025,0.044489,0.051354,31.399609
min,18.000000,1.000000,12.865200,77.484200,560001.000000
25%,23.000000,2.000000,12.936900,77.565275,560010.750000
50%,24.000000,3.000000,12.977000,77.592100,560033.500000
75%,26.000000,4.000000,12.997025,77.630900,560068.000000
max,33.000000,6.000000,13.102000,77.758200,560109.000000


Saving the CSV to be used in PowerBI

In [12]:
df_online_food.to_csv('New_online_food.csv', index=False)